# **Báo cáo bài tập lớn môn Học máy: Quora Insincere Questions Classification**
**Sinh viên:** Phan Hải Anh<br>
**MSSV:** 19021213<br>

# **Mô tả bài toán**

**Quora Insincere Question Classification** là một bài toán của **Quora** đặt ra, sử dụng sự trợ giúp từ cộng đồng, giúp họ phân loại những câu hỏi không chân thành. Nhiệm vụ của bài toán là sử dụng tập dữ liệu mà **Quora** cung cấp để phân loại ra những câu hỏi mang hàm ý không chân thành, mang nội dung xấu độc, gây hiểu lầm.

- **Đầu vào**: Câu hỏi dưới dạng văn bản (plain text)
- **Đầu ra**: Yes/No (insincere (1) or not (0))

- Tiền xử lý bộ dữ liệu
    - Bỏ đi các biểu thức toán học và các đường dẫn liên kết
    - Bỏ đi các ký tự đặc biệt, các chữ số
    - Sửa những từ sai chính tả và extend các từ viết tắt
    - Bỏ đi stop_words trong câu
    - Lemming các từ có trong câu (có thể hiểu là chuyển các từ về dạng nguyên thể)
    
> **Mục đích**: Loại bỏ đi các từ gây nhiễu cho bộ dữ liệu và mở rộng độ bao phủ của tập Vocab với bộ dữ liệu
- Xây dựng tập vocab: 
    - Sử dụng các file trong embeddings.zip mà quora cung cấp để xây dựng tập vocab 
    - Các file embeddings có dạng text, mỗi dòng chứa 1 word và đi kèm đó là vector của nó đã được pretrained
    - Xây tập vocab từ các word có trong từ điển và vector đi kèm với chúng
    - Kiểm tra độ phủ của vocab xây dựng dựa trên tập embeddings với dữ liệu được cho
- Xây dựng ma trận embeddings:
    - Sử dụng file pretrained để tạo ra ma trận embeddings dựa vào các từ có trong tập vocab
    - Lấy các từ để lookup vector descriptor của các từ đó và xây ma trận
- Tạo model và huấn luyện dự đoán
- Kết hợp kết quả của nhiều model với các ma trận embeddings đã xây dựng và cho ra kết quả tốt nhất

# **1. Khảo sát dữ liệu**

Xác định các tập ```train```, ```test``` và khởi tạo một tập chứa dữ liệu của cả 2 để về sau check độ bao phủ của vocab với tập dữ liệu đã sinh

**Tập ```train```: **
Ở trong bộ dữ liệu có 3 trường, và khả năng dữ liệu train nằm ở trường **```question_text```** <br>
Vậy ta sẽ thử khảo sát dữ liệu trong trường **```question_text```**
Có thể thấy rằng tập train có 3 trường dữ liệu
- **```qid```**: định danh id cho câu hỏi
- **```question_text```**: nội dung câu hỏi
- **```target```**: phân lớp câu hỏi
    - 0: câu hỏi mang tính chất chân thành (sincere)
    - 1: câu hỏi không mang tính chất chân thành (insincere)
    
Khá may mắn khi tập dữ liệu **Quora** cung cấp không có bất kì trường dữ liệu nào có giá trị bất thường (```null```, ```none```, ```missing```)
    
Tập dữ liệu train bao gồm **1306122** dòng dữ liệu, gần **1.31 triệu dòng**, khá lớn. 

Có thể thấy trong trường target có 2 giá trị là ```0``` và ```1```<br>
Dự đoán thì những câu hỏi đánh dấu ```0``` là những câu hỏi sincere, tương tự ```1``` là insincere<br>
Vậy có thể xem như đây là bài toán phân lớp nhị phân<br>
Nhưng vẫn cần phải có đánh giá sâu hơn về bộ dữ liệu này, nhìn nhanh qua có thể thấy bộ dữ liệu train được cho khá là mất cân bằng

Qua biểu đồ có thể dễ dàng nhận thấy tập dữ liệu train bị mất cân bằng nhiều:
- Lớp câu hỏi sincere có **1225312** (chiếm **93.81%**) số lượng dữ liệu
- Lớp câu hỏi insincere có **80810** (chiếm **6.19%**) số lượng dữ liệu

Có thể thấy tập dữ liệu bị lệch nhiều (lên tới **15 lần**) cho nên cần phải sử dụng metrics f1_score để đánh giá độ hiệu quả của mô hình hơn là so **accuracy**<br>
Do **f1_score** quan tâm đến phân bố của dữ liệu nên trong bài này nó sẽ mang nhiều ý nghĩa hơn là **accuracy**.<br>
Và trong cuộc thi **Kaggle** cũng bảo là sử dụng **f1_score** thay vì **accuracy**

**Tập ```test```**:
Ở trong bộ dữ liệu test thì chỉ có 2 trường dữ liệu là qid đại diện cho id của câu hỏi
Và question_text đại diện cho dữ liệu test
Cũng may mắn khi mà ở trong tập df_test lại không có dữ liệu kì lạ nào cả (null, none, missing)

Mỗi lần shuffle lại cho ra một kết quả khác, nhưng nhìn chung qua vài lần thực nghiệm quan sát, em nhận thấy trong tập dữ liệu ở trên:

Có nhiều câu mà các từ trong đó viết sai chính tả, lẫn lộn giữa tiếng Anh-Anh và tiếng Anh-Mỹ
Nhiều câu sử dụng các từ viết tắt: He's mà đáng nhẽ nên là He is.
Có một số từ viết tắt cho tên các tổ chức, hay các chuẩn mã hoá kiểu RSA, DES/3DES, ...
Nhiều câu sử dụng các ký tự đặc biệt cho nên sẽ gây ảnh hưởng lớn tới mô hình dự đoán
Về mặt nội dung hàm ý thì có thể không ảnh hưởng nhưng sẽ ảnh hưởng khi xâp tập từ điển, cùng một ý nghĩa nhưng các từ lại được tính nhiều lần giá trị

Thấy rằng, bộ dữ liệu đã cho có khá nhiều nhiễu => cần loại bỏ nhiều nhiều nhất có thể

Cách xử lý

Loại bỏ stopwords có trong câu
Stem các từ có trong câu (driver, driving, driven, drove -> drive)
Chuẩn hoá các từ về dạng lowercase
Loại bỏ các ký tự đặc biệt
Loại bỏ các đường dẫn liên kết, loại bỏ các công thức toán học, ...

### **Nhận xét**
Nhưng có thể thấy một điều là bộ dữ liệu **train** và **test** dữ liệu đầu vào đều ở dạng plain text và mô hình của chúng ta sẽ không thể hiểu nổi<br>
Vậy hướng giải quyết ở đây em nghĩ là là sử dụng mã hoá **one_hot** đưa dữ liệu về dạng binary để cho mô hình có thể hiểu được

In [ ]:
import pandas as pd 
import seaborn as sns
import re
import gc
import os
import numpy as np
import operator
from wordcloud import WordCloud, STOPWORDS
from tqdm import tqdm
import matplotlib.pyplot as plt
pd_ctx = pd.option_context('display.max_colwidth', 100)

import nltk
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer

from gensim.models import KeyedVectors

import tensorflow as tf

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D,GRU
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, SpatialDropout1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

df_train = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
df_test = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
quora_data = df_train['question_text'].append(df_test['question_text'])

# KHẢO SÁT DỮ LIỆU TRONG TẬP TRAIN

print("\033[1mTrain set info\033[0m")
print(df_train.info())

# Kiểm tra các trường dữ liệu của câu hỏi được đánh sincere
print("\033[1mSincere Questions: \033[0m")
display(df_train[df_train['target']==0].head())
# Kiểm tra các trường dữ liệu của câu hỏi được đánh sincere
print("\033[1mInsincere Questions: \033[0m")
display(df_train[df_train['target']==1].head())

df_train.target.value_counts()

pos_len = len(df_train[df_train['target'] == 1])
neg_len = len(df_train[df_train['target'] == 0])
total = len(df_train)
print("\033[1mTotal = \033[0m", total)
print("\033[1mSincere questions:\033[0m {neg} ({percent: .2f}% )".format(neg = neg_len, percent = neg_len / total * 100))
print("\033[1mInsincere questions:\033[0m {pos} ({percent: .2f}% )".format(pos = pos_len, percent = pos_len / total * 100))
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(['sincere', 'insincere'], df_train.target.value_counts())
plt.show()

#KHẢO SÁT DỮ LIỆU TRONG TẬP TEST

df_test.info()
# Shuffle tập train để kiểm tra những giá trị ngẫu nhiên
train = df_train.sample(frac=1).reset_index(drop=True)
display(train.sample(n=10, random_state=344))

# **2. Tiền xử lý dữ liệu**

Với bài toán liên quan đến ngôn ngữ thì hiệu quả của mô hình phụ thuộc lớn vào việc tiền xử lý dữ liệu, tức loại bỏ đi nhiễu ở trong bộ dữ liệu.<br>
Và phương pháp phổ biến đối với các bài toán liên quan đến ngôn ngữ được trình bày ở dưới như sau

**Cách làm:**<br>
Sử dụng thư viện xử lý ngôn ngữ tự nhiên nltk trong việc preprocess data<br>
Ở đây yêu cần phải sử dụng wordnet, một thư viện từ đồng nghĩa, trái nghĩa, nltk punkt được yêu cầu để tokenize words<br>
- Bước 1: ```clean_tag()```: loại bỏ đi các biểu thức toán học, các địa chỉ liên kết
- Bước 2: ```clean_puncts()```: loại bỏ đi các ký tự đặc biệt có trong câu
- Bước 3: ```correct_misspell()```: trong bộ dữ liệu của **Quora** do đây là các câu hỏi của người dùng nên không tránh khỏi việc gõ sai, hay là có những từ không chuẩn ví dụ là lẫn lộn giữa tiếng Anh-Anh với tiếng Anh-Mỹ cho nên cần phải fix những từ này và đưa nó về dạng chuẩn
- Bước 4: ```remove_stopwords()```: loại bỏ đi các stopwords có trong câu( những từ, chữ dạng *'do'*, *'does'*, *'did'*, *'should'*, ...)
- Bước 5: ```clean_contractions()```: chuyển những từ viết tắt về dạng đầy đủ vốn có
- Bước 6: ```lemming_words()```: lemming các từ về dạng nguyên bản của nó


**Vấn đề:**<br>
Tuy nhiên đời không như là mơ, cuộc thi này của **Quora** không cho phép sử dụng Internet nên việc lemming word bằng cách lookup từ điển này sẽ không hoạt động<br>
**Giải pháp:**<br>
Cần tìm ra một phương pháp lemming words mới

In [ ]:
# CÁC HÀM XỬ LÝ

def clean_tag(x):
  if '[math]' in x:
    x = re.sub('\[math\].*?math\]', 'MATH EQUATION', x) #replacing with [MATH EQUATION]
    
  if 'http' in x or 'www' in x:
    x = re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', 'URL', x) #replacing with [url]
  return x

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′',  '█', '…', '“', '★', '”', '–', '●', '►', '−', '¢', '¬', '░', '¡', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '¸', '⋅', '‘', '∞',  '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '・', '╦', '╣', '╔', '╗', '▬', '❤', '≤', '‡', '√', '◄', '━', '⇒', '▶', '≥', '╝', '♡', '◊', '。', '✈', '≡', '☺', '✔', '↵', '≈', '✓', '♣', '☎', '℃', '◦', '└', '‟', '～', '！', '○',  '◆', '№', '♠', '▌', '✿', '▸', '⁄', '□', '❖', '✦', '．', '÷', '｜', '┃', '／', '￥', '╠', '↩', '✭', '▐', '☼', '☻', '┐', '├', '«', '∼', '┌', '℉', '☮', '฿', '≦', '♬', '✧', '〉', '－', '⌂', '✖', '･', '◕', '※', '‖', '◀', '‰', '\x97', '↺', '∆', '┘', '┬', '╬', '،', '⌘', '⊂', '＞', '〈', '⎙', '？', '☠', '⇐', '▫', '∗', '∈', '≠', '♀', '♔', '˚', '℗', '┗', '＊', '┼', '❀', '＆', '∩', '♂', '‿', '∑', '‣', '➜', '┛', '⇓', '☯', '⊖', '☀', '┳', '；', '∇', '⇑', '✰', '◇', '♯', '☞', '´', '↔', '┏', '｡', '◘', '∂', '✌', '♭', '┣', '┴', '┓', '✨', '\xa0', '˜', '❥', '┫', '℠', '✒', '［', '∫', '\x93', '≧', '］',  '\x94', '∀', '♛', '\x96', '∨', '◎', '↻', '⇩', '＜', '≫', '✩', '✪', '♕', '؟', '₤', '☛', '╮', '␊', '＋', '┈', '％',  '╋', '▽', '⇨', '┻', '⊗', '￡', '।', '▂', '✯', '▇', '＿', '➤', '✞', '＝', '▷', '△', '◙', '▅', '✝', '∧', '␉', '☭', '┊', '╯', '☾', '➔', '∴', '\x92', '▃', '↳', '＾', '׳', '➢', '╭', '➡', '＠', '⊙', '☢', '˝', '∏', '„', '∥', '❝', '☐',  '▆', '╱', '⋙', '๏', '☁', '⇔', '▔', '\x91', '➚', '◡', '╰', '\x85', '♢', '˙', '۞', '✘', '✮', '☑', '⋆', 'ⓘ', '❒', '☣', '✉', '⌊', '➠', '∣', '❑', '◢', 'ⓒ', '\x80', '〒', '∕', '▮', '⦿', '✫', '✚', '⋯', '♩', '☂', '❞', '‗', '܂', '☜','‾', '✜', '╲', '∘', '⟩', '＼', '⟨', '·', '✗', '♚', '∅', 'ⓔ', '◣', '͡', '‛', '❦', '◠', '✄', '❄', '∃', '␣', '≪', '｢',  '≅', '◯', '☽', '∎', '｣', '❧', '̅', 'ⓐ', '↘', '⚓', '▣', '˘', '∪', '⇢', '✍', '⊥', '＃', '⎯', '↠', '۩', '☰', '◥', '⊆', '✽', '⚡', '↪', '❁', '☹', '◼', '☃', '◤', '❏', 'ⓢ', '⊱', '➝', '̣', '✡', '∠', '｀', '▴', '┤', '∝', '♏', 'ⓐ',  '✎', ';', '␤', '＇', '❣', '✂', '✤', 'ⓞ', '☪', '✴', '⌒', '˛', '♒', '＄', '✶', '▻', 'ⓔ', '◌', '◈', '❚', '❂', '￦', '◉', '╜', '̃', '✱', '╖', '❉', 'ⓡ', '↗', 'ⓣ', '♻', '➽', '׀', '✲', '✬', '☉', '▉', '≒', '☥', '⌐', '♨', '✕', 'ⓝ',   '⊰', '❘', '＂', '⇧', '̵', '➪', '▁', '▏', '⊃', 'ⓛ', '‚', '♰', '́', '✏', '⏑', '̶', 'ⓢ', '⩾', '￠', '❍', '≃', '⋰', '♋',  '､', '̂', '❋', '✳', 'ⓤ', '╤', '▕', '⌣', '✸', '℮', '⁺', '▨', '╨', 'ⓥ', '♈', '❃', '☝', '✻', '⊇', '≻', '♘', '♞',  '◂', '✟', '⌠', '✠', '☚', '✥', '❊', 'ⓒ', '⌈', '❅', 'ⓡ', '♧', 'ⓞ', '▭', '❱', 'ⓣ', '∟', '☕', '♺', '∵', '⍝', 'ⓑ',  '✵', '✣', '٭', '♆', 'ⓘ', '∶', '⚜', '◞', '்', '✹', '➥', '↕', '̳', '∷', '✋', '➧', '∋', '̿', 'ͧ', '┅', '⥤', '⬆', '⋱', '☄', '↖', '⋮', '۔', '♌', 'ⓛ', '╕', '♓', '❯', '♍', '▋', '✺', '⭐', '✾', '♊', '➣', '▿', 'ⓑ', '♉', '⏠', '◾', '▹',  '⩽', '↦', '╥', '⍵', '⌋', '։', '➨', '∮', '⇥', 'ⓗ', 'ⓓ', '⁻', '⎝', '⌥', '⌉', '◔', '◑', '✼', '♎', '♐', '╪', '⊚',  '☒', '⇤', 'ⓜ', '⎠', '◐', '⚠', '╞', '◗', '⎕', 'ⓨ', '☟', 'ⓟ', '♟', '❈', '↬', 'ⓓ', '◻', '♮', '❙', '♤', '∉', '؛',  '⁂', 'ⓝ', '־', '♑', '╫', '╓', '╳', '⬅', '☔', '☸', '┄', '╧', '׃', '⎢', '❆', '⋄', '⚫', '̏', '☏', '➞', '͂', '␙', 'ⓤ', '◟', '̊', '⚐', '✙', '↙', '̾', '℘', '✷', '⍺', '❌', '⊢', '▵', '✅', 'ⓖ', '☨', '▰', '╡', 'ⓜ', '☤', '∽', '╘', '˹', '↨', '♙', '⬇', '♱', '⌡', '⠀', '╛', '❕', '┉', 'ⓟ', '̀', '♖', 'ⓚ', '┆', '⎜', '◜', '⚾', '⤴', '✇', '╟', '⎛',    '☩', '➲', '➟', 'ⓥ', 'ⓗ', '⏝', '◃', '╢', '↯', '✆', '˃', '⍴', '❇', '⚽', '╒', '̸', '♜', '☓', '➳', '⇄', '☬', '⚑',   '✐', '⌃', '◅', '▢', '❐', '∊', '☈', '॥', '⎮', '▩', 'ு', '⊹', '‵', '␔', '☊', '➸', '̌', '☿', '⇉', '⊳', '╙', 'ⓦ',    '⇣', '｛', '̄', '↝', '⎟', '▍', '❗', '״', '΄', '▞', '◁', '⛄', '⇝', '⎪', '♁', '⇠', '☇', '✊', 'ி', '｝', '⭕', '➘',   '⁀', '☙', '❛', '❓', '⟲', '⇀', '≲', 'ⓕ', '⎥', '\u06dd', 'ͤ', '₋', '̱', '̎', '♝', '≳', '▙', '➭', '܀', 'ⓖ', '⇛', '▊',  '⇗', '̷', '⇱', '℅', 'ⓧ', '⚛', '̐', '̕', '⇌', '␀', '≌', 'ⓦ', '⊤', '̓', '☦', 'ⓕ', '▜', '➙', 'ⓨ', '⌨', '◮', '☷',    '◍', 'ⓚ', '≔', '⏩', '⍳', '℞', '┋', '˻', '▚', '≺', 'ْ', '▟', '➻', '̪', '⏪', '̉', '⎞', '┇', '⍟', '⇪', '▎', '⇦', '␝', '⤷', '≖', '⟶', '♗', '̴', '♄', 'ͨ', '̈', '❜', '̡', '▛', '✁', '➩', 'ா', '˂', '↥', '⏎', '⎷', '̲', '➖', '↲', '⩵', '̗', '❢', 
        '≎', '⚔', '⇇', '̑', '⊿', '̖', '☍', '➹', '⥊', '⁁', '✢']

def clean_punct(x):
  x = str(x)
  for punct in puncts:
    if punct in x:
      x = x.replace(punct, ' ')
    return x

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'bitcoin', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization', 
                'electroneum':'bitcoin','nanodegree':'degree','hotstar':'star','dream11':'dream','ftre':'fire','tensorflow':'framework','unocoin':'bitcoin',
                'lnmiit':'limit','unacademy':'academy','altcoin':'bitcoin','altcoins':'bitcoin','litecoin':'bitcoin','coinbase':'bitcoin','cryptocurency':'cryptocurrency',
                'simpliv':'simple','quoras':'quora','schizoids':'psychopath','remainers':'remainder','twinflame':'soulmate','quorans':'quora','brexit':'demonetized', 'iiest':'institute','dceu':'comics','pessat':'exam','uceed':'college','bhakts':'devotee','boruto':'anime','cryptocoin':'bitcoin','blockchains':'blockchain','fiancee':'fiance','redmi':'smartphone','oneplus':'smartphone','qoura':'quora','deepmind':'framework','ryzen':'cpu','whattsapp':'whatsapp',
                'undertale':'adventure','zenfone':'smartphone','cryptocurencies':'cryptocurrencies','koinex':'bitcoin','zebpay':'bitcoin','binance':'bitcoin','whtsapp':'whatsapp',
                'reactjs':'framework','bittrex':'bitcoin','bitconnect':'bitcoin','bitfinex':'bitcoin','yourquote':'your quote','whyis':'why is','jiophone':'smartphone',
                'dogecoin':'bitcoin','onecoin':'bitcoin','poloniex':'bitcoin','7700k':'cpu','angular2':'framework','segwit2x':'bitcoin','hashflare':'bitcoin','940mx':'gpu',
                'openai':'framework','hashflare':'bitcoin','1050ti':'gpu','nearbuy':'near buy','freebitco':'bitcoin','antminer':'bitcoin','filecoin':'bitcoin','whatapp':'whatsapp',
                'empowr':'empower','1080ti':'gpu','crytocurrency':'cryptocurrency','8700k':'cpu','whatsaap':'whatsapp','g4560':'cpu','payymoney':'pay money',
                'fuckboys':'fuck boys','intenship':'internship','zcash':'bitcoin','demonatisation':'demonetization','narcicist':'narcissist','mastuburation':'masturbation',
                'trignometric':'trigonometric','cryptocurreny':'cryptocurrency','howdid':'how did','crytocurrencies':'cryptocurrencies','phycopath':'psychopath',
                'bytecoin':'bitcoin','possesiveness':'possessiveness','scollege':'college','humanties':'humanities','altacoin':'bitcoin','demonitised':'demonetized',
                'brasília':'brazilia','accolite':'accolyte','econimics':'economics','varrier':'warrier','quroa':'quora','statergy':'strategy','langague':'language','splatoon':'game','7600k':'cpu','gate2018':'gate 2018','in2018':'in 2018','narcassist':'narcissist','jiocoin':'bitcoin','hnlu':'hulu','7300hq':'cpu','weatern':'western','interledger':'blockchain','deplation':'deflation', 'cryptocurrencies':'cryptocurrency', 'bitcoin':'blockchain cryptocurrency',}

def correct_mispell(x):
  words = x.split()
  for i in range(0, len(words)):
    if mispell_dict.get(words[i]) is not None:
      words[i] = mispell_dict.get(words[i])
    elif mispell_dict.get(words[i].lower()) is not None:
      words[i] = mispell_dict.get(words[i].lower())
        
  words = " ".join(words)
  return words

def remove_stopwords(x):
  x = [word for word in x.split() if word not in STOPWORDS]
  x = ' '.join(x)
  return x

contraction_mapping = {
 "I'm": 'I am', "I'm'a": 'I am about to', "I'm'o": 'I am going to',"I've": 'I have', "I'll": 'I will', "I'll've": 'I will have', "I'd": 'I would', "I'd've": 'I would have', "i'm": 'i am', "i'm'a": 'i am about to', "i'm'o": 'i am going to', "i've": 'i have', "i'll": 'i will', "i'll've": 'i will have', "i'd": 'i would', "i'd've": 'i would have', 'Whatcha': 'What are you', 'whatcha': 'what are you', "amn't": 'am not', "ain't": 'are not', "aren't": 'are not', "'cause": 'because', "can't": 'can not', "can't've": 'can not have', "could've": 'could have', "couldn't": 'could not', "couldn't've": 'could not have', "daren't": 'dare not', "daresn't": 'dare not', "dasn't": 'dare not', "didn't": 'did not', 'didn’t': 'did not', "don't": 'do not', 'don’t': 'do not', "doesn't": 'does not', "e'er": 'ever', "everyone's": 'everyone is', 'finna': 'fixing to', 'gimme': 'give me', "gon't": 'go not', 'gonna': 'going to', 'gotta': 'got to', "hadn't": 'had not', "hadn't've": 'had not have', "hasn't": 'has not', "haven't": 'have not', "he've": 'he have', "he's": 'he is', "he'll": 'he will', "he'll've": 'he will have', "he'd": 'he would', "he'd've": 'he would have', "here's": 'here is', "how're": 'how are', "how'd": 'how did', "how'd'y": 'how do you', "how's": 'how is', "how'll": 'how will', "isn't": 'is not', "it's": 'it is', "'tis": 'it is', "'twas": 'it was', "it'll": 'it will', "it'll've": 'it will have', "it'd": 'it would', "it'd've": 'it would have', 'kinda': 'kind of', "let's": 'let us', 'luv': 'love', "ma'am": 'madam', "may've": 'may have', "mayn't": 'may not', "might've": 'might have', "mightn't": 'might not', "mightn't've": 'might not have', "must've": 'must have', "mustn't": 'must not', "mustn't've": 'must not have', "needn't": 'need not', "needn't've": 'need not have', "ne'er": 'never', "o'": 'of', "o'clock": 'of the clock', "ol'": 'old', "oughtn't": 'ought not', "oughtn't've": 'ought not have', "o'er": 'over', "shan't": 'shall not', "sha'n't": 'shall not', "shalln't": 'shall not', "shan't've": 'shall not have', "she's": 'she is', "she'll": 'she will', "she'd": 'she would', "she'd've": 'she would have', "should've": 'should have', "shouldn't": 'should not', "shouldn't've": 'should not have', "so've": 'so have', "so's": 'so is', "somebody's": 'somebody is', "someone's": 'someone is', "something's": 'something is', 'sux': 'sucks', "that're": 'that are', "that's": 'that is', "that'll": 'that will', "that'd": 'that would', "that'd've": 'that would have', 'em': 'them', "there're": 'there are', "there's": 'there is', "there'll": 'there will', "there'd": 'there would', "there'd've": 'there would have', "these're": 'these are', "they're": 'they are', "they've": 'they have', "they'll": 'they will', "they'll've": 'they will have', "they'd": 'they would', "they'd've": 'they would have', "this's": 'this is', "those're": 'those are', "to've": 'to have', 'wanna': 'want to', "wasn't": 'was not', "we're": 'we are', "we've": 'we have', "we'll": 'we will', "we'll've": 'we will have', "we'd": 'we would', "we'd've": 'we would have', "weren't": 'were not', "what're": 'what are', "what'd": 'what did', "what've": 'what have', "what's": 'what is', "what'll": 'what will', "what'll've": 'what will have', "when've": 'when have', "when's": 'when is', "where're": 'where are', "where'd": 'where did', "where've": 'where have', "where's": 'where is', "which's": 'which is', "who're": 'who are', "who've": 'who have', "who's": 'who is', "who'll": 'who will', "who'll've": 'who will have', "who'd": 'who would', "who'd've": 'who would have', "why're": 'why are', "why'd": 'why did', "why've": 'why have', "why's": 'why is', "will've": 'will have', "won't": 'will not', "won't've": 'will not have', "would've": 'would have', "wouldn't": 'would not', "wouldn't've": 'would not have',
 "y'all": 'you all', "y'all're": 'you all are', "y'all've": 'you all have', "y'all'd": 'you all would', "y'all'd've": 'you all would have', "you're": 'you are', "you've": 'you have', "you'll've": 'you shall have', "you'll": 'you will', "you'd": 'you would', "you'd've": 'you would have', 'jan.': 'january', 'feb.': 'february', 'mar.': 'march', 'apr.': 'april', 'jun.': 'june', 'jul.': 'july', 'aug.': 'august', 'sep.': 'september', 'oct.': 'october', 'nov.': 'november', 'dec.': 'december', 'I’m': 'I am', 'I’m’a': 'I am about to', 'I’m’o': 'I am going to', 'I’ve': 'I have', 'I’ll': 'I will', 'I’ll’ve': 'I will have', 'I’d': 'I would', 'I’d’ve': 'I would have', 'i’m': 'i am', 'i’m’a': 'i am about to', 'i’m’o': 'i am going to', 'i’ve': 'i have', 'i’ll': 'i will', 'i’ll’ve': 'i will have', 'i’d': 'i would', 'i’d’ve': 'i would have', 'amn’t': 'am not', 'ain’t': 'are not','aren’t': 'are not','’cause': 'because','can’t': 'can not','can’t’ve': 'can not have','could’ve': 'could have','couldn’t': 'could not','couldn’t’ve': 'could not have','daren’t': 'dare not','daresn’t': 'dare not','dasn’t': 'dare not','doesn’t': 'does not','e’er': 'ever','everyone’s': 'everyone is','gon’t': 'go not','hadn’t': 'had not','hadn’t’ve': 'had not have','hasn’t': 'has not','haven’t': 'have not','he’ve': 'he have','he’s': 'he is','he’ll': 'he will','he’ll’ve': 'he will have','he’d': 'he would','he’d’ve': 'he would have','here’s': 'here is','how’re': 'how are','how’d': 'how did','how’d’y': 'how do you','how’s': 'how is','how’ll': 'how will',
 'isn’t': 'is not','it’s': 'it is','’tis': 'it is','’twas': 'it was','it’ll': 'it will','it’ll’ve': 'it will have','it’d': 'it would','it’d’ve': 'it would have','let’s': 'let us','ma’am': 'madam','may’ve': 'may have','mayn’t': 'may not','might’ve': 'might have','mightn’t': 'might not','mightn’t’ve': 'might not have','must’ve': 'must have','mustn’t': 'must not','mustn’t’ve': 'must not have','needn’t': 'need not','needn’t’ve': 'need not have','ne’er': 'never','o’': 'of','o’clock': 'of the clock','ol’': 'old','oughtn’t': 'ought not','oughtn’t’ve': 'ought not have','o’er': 'over',
 'shan’t': 'shall not','sha’n’t': 'shall not','shalln’t': 'shall not','shan’t’ve': 'shall not have','she’s': 'she is','she’ll': 'she will','she’d': 'she would','she’d’ve': 'she would have','should’ve': 'should have','shouldn’t': 'should not','shouldn’t’ve': 'should not have','so’ve': 'so have','so’s': 'so is','somebody’s': 'somebody is','someone’s': 'someone is','something’s': 'something is','that’re': 'that are','that’s': 'that is','that’ll': 'that will','that’d': 'that would','that’d’ve': 'that would have','there’re': 'there are','there’s': 'there is','there’ll': 'there will','there’d': 'there would','there’d’ve': 'there would have','these’re': 'these are','they’re': 'they are','they’ve': 'they have','they’ll': 'they will','they’ll’ve': 'they will have','they’d': 'they would','they’d’ve': 'they would have','this’s': 'this is','those’re': 'those are','to’ve': 'to have','wasn’t': 'was not','we’re': 'we are','we’ve': 'we have','we’ll': 'we will','we’ll’ve': 'we will have','we’d': 'we would','we’d’ve': 'we would have','weren’t': 'were not','what’re': 'what are','what’d': 'what did','what’ve': 'what have','what’s': 'what is','what’ll': 'what will','what’ll’ve': 'what will have','when’ve': 'when have','when’s': 'when is','where’re': 'where are','where’d': 'where did','where’ve': 'where have','where’s': 'where is','which’s': 'which is','who’re': 'who are','who’ve': 'who have','who’s': 'who is','who’ll': 'who will','who’ll’ve': 'who will have','who’d': 'who would','who’d’ve': 'who would have','why’re': 'why are','why’d': 'why did','why’ve': 'why have','why’s': 'why is','will’ve': 'will have','won’t': 'will not','won’t’ve': 'will not have','would’ve': 'would have','wouldn’t': 'would not','wouldn’t’ve': 'would not have','y’all': 'you all','y’all’re': 'you all are','y’all’ve': 'you all have','y’all’d': 'you all would','y’all’d’ve': 'you all would have','you’re': 'you are','you’ve': 'you have','you’ll’ve': 'you shall have','you’ll': 'you will','you’d': 'you would','you’d’ve': 'you would have'}

def clean_contractions(text):
#     text = text.lower()
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    
    text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
    return text

lemmatizer = WordNetLemmatizer()
def lemma_text(x):
    x = x.split()
    x = [lemmatizer.lemmatize(word) for word in x]
    x = ' '.join(x)
    return x

def data_cleaning(x):
  x = clean_tag(x)
  x = clean_punct(x)
  x = correct_mispell(x)
  x = remove_stopwords(x)
  x = clean_contractions(x)
  x = lemma_text(x)
  return x

def Preprocess(doc):
    corpus=[]
    for text in tqdm(doc):
        text=clean_contractions(text)
        text=correct_mispell(text)
        text=re.sub(r'[^a-z0-9A-Z]'," ",text)
        text=re.sub(r'[0-9]{1}',"#",text)
        text=re.sub(r'[0-9]{2}','##',text)
        text=re.sub(r'[0-9]{3}','###',text)
        text=re.sub(r'[0-9]{4}','####',text)
        text=re.sub(r'[0-9]{5,}','#####',text)
        corpus.append(text)
    return corpus


In [ ]:
# TIỀN XỬ LÝ CÁC TẬP DỮ LIỆU

tqdm.pandas(desc="progress-bar")
df_test['question_text_cleaned'] = df_test['question_text'].progress_map(lambda x: data_cleaning(x))
df_train['question_text_cleaned'] = df_train['question_text'].progress_map(lambda x: data_cleaning(x))

df_train.head(5)

# **3. Build tập vocab**

Ở đây em sẽ cố định sử dụng thằng **Google News** làm tiêu chuẩn<br>
Kiểm tra word embeddings đã giải nén với **Google News** đính kèm trong file embeddings.zip<br>
Định nghĩa các hàm để build vocab từ file **Google News** và hàm kiểm tra độ bao phủ của vocab đối với tập dữ liệu đã được preprocess<br>
&nbsp;<br>
Trong bài này, em sẽ giải nén tệp embedding để sử dụng tập các vector với các từ cho sẵn<br>
Các từ trong file embeddings đều đính kèm một vector, và em sẽ xây ma trận embeddings dựa vào file embeddings đó

File **Google News** có kích thước khá nhỏ: ```3.4G``` nhưng nó lại ở dạng binary của word embeddings nên đã có thư viện KeyedVector hỗ trợ và việc load khá nhanh nên em sẽ sử dụng thằng **Google News** này làm tham chiếu cho các embeddings khác trong tương lai<br>
Ở dưới em sẽ định nghĩa các hàm build ```vocab``` và check độ phủ<br>
Do embeddings là file có word đính kèm vector mô tả cho nó nên em sẽ lookup các từ có trong câu với file embeddings và tìm vector của chúng

Xây dựng tập vocab và kiểm tra độ bao phủ của tập vocab với tập vocab của pretrained model từ Google News<br>
Trong block code này em sẽ xây tập vocab dựa trên bộ dữ liệu được xử lý với hàm preprocess thứ nhất là ```data_clean()```

Với hàm preprocess thứ nhất là ```data_clean()``` thì thầy có thể thấy kết quả độ bao phủ đạt **81.04%**, hơi thấp một chút dù dữ liệu đã được xử lý từ trước<br>
Nhưng tệ hại là tập vocab chỉ bao phủ **25.40%** corpus<br>
**Nhận xét:**<br>
Có thể là do việc đụng chạm tới nhiều từ trong câu khi loại bỏ nó, và sửa lỗi sai chính tả. Điều này đã khiến giảm đi đáng kể số lượng các từ được tìm thấy trong file pretrained\
**Giải pháp:**<br>
Sử dụng một hàm Preprocess mới mà ít đụng chạm tới các từ, chỉ xử lý một cách đơn giản các câu

Trong block code này, em sẽ sử dụng hàm Preprocess thứ 2 (hàm process đơn giản hơn)
Như thầy có thể thấy hiệu quả được cải thiện một cách rõ rệt:
- Độ phủ của vocab với corpus tăng gấp **2.5 lần** lên **61.37%**
- Độ phủ tăng lên **90.81%**, một con số khá ấn tượng

Nếu như chuyển các từ về dạng ```lowercase``` thì 2 con số tương đương là **38.85%** với corpus và **89.47%** với vocab.<br>
Người Anh đặt cái tôi của họ rất cao, nên trong tất cả các câu văn của họ thì từ 'I' (chỉ bản thân người nói) luôn được đặt ở trạng thái in hoa ```uppercase```nên nếu chúng ta chuẩn hoá các từ về ```lowercase``` thì sẽ không lookup được từ đó trong file pretrained embeddings và sẽ gây ra hiện tượng thiếu sót trong ma trận embeddings mà chúng ta chuẩn bị xây dựng

Tập pretrained **Google News** dù độ phủ khá cao nhưng số lượng từ nằm ngoài tập embeddings khá là lớn khi lên tới 99931 từ<br>
Và cần phải có sự cải thiện về số lượng từ ```out_of_vocab``` nếu không thì hiệu quả của model sẽ kém.

In [ ]:
### unzipping all the pretrained embeddings
!unzip ../input/quora-insincere-questions-classification/embeddings.zip

!du -h ./

### Loading tập Google News Pretrained Embeddings vào bộ nhớ
file_name="./GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin"
model_embed=KeyedVectors.load_word2vec_format(file_name,binary=True)
# model_embed = load_embed1('./glove.840B.300d/glove.840B.300d.txt')

### Xây dựng tập từ vựng dựa trên dữ liệu của tập Google News
def vocab_build(corpus):
    vocab={}
    for text in tqdm(corpus):
        for word in text.split():
            try:
                vocab[word]+=1
            except KeyError:
                vocab[word]=1
    return vocab


### Kiểm tra tập Vocabulary xem tập vocab đó bao phủ bao nhiêu phần trăm tập dữ liệu của mình
def check_voc(vocab,model):
    embed_words=[]
    out_vocab={}
    total_words=0
    total_text=0
    for i in tqdm(vocab):
        try:
            vec=model[i]
            embed_words.append(vec)
            total_words+=vocab[i]
        except KeyError:
            out_vocab[i]=vocab[i]
            total_text+=vocab[i]
    print("The {:.2f}% of vocabularies have Covered of corpus".format(100*len(embed_words)/len(vocab)))
    print("The {:.2f}% of total text had coverded ".format((100*total_words/(total_words+total_text))))
    return out_vocab

### xây tập vocab và kiểm tra độ phủ của tập vocab với dữ liệu đã được xử lý
total_text=pd.concat([df_train.question_text_cleaned,df_test.question_text_cleaned])
vocabulary=vocab_build(total_text)
oov=check_voc(vocabulary,model_embed) #oov: out of vocab

df_test['question_text_cleaned_2'] = Preprocess(df_test['question_text'])
df_train['question_text_cleaned_2'] = Preprocess(df_train['question_text'])
total_text_2=pd.concat([df_train.question_text_cleaned_2,df_test.question_text_cleaned_2])
vocabulary2=vocab_build(total_text_2)
oov2=check_voc(vocabulary2, model_embed)

sort_oov=dict(sorted(oov2.items(), key=operator.itemgetter(1),reverse=True))
dict(list(sort_oov.items())[:50])

del oov, oov2,sort_oov,total_text,total_text_2
gc.collect()

def get_word_index(vocab):
    word_index=dict((w,i+1) for i,w in enumerate(vocab.keys()))
    return word_index
def fit_one_hot(word_index,corpus):
    sent=[]
    for text in tqdm(corpus):
        li=[]
        for word in text.split():
            try:
                li.append(word_index[word])
            except KeyError:
                li.append(0)
        sent.append(li)
    return sent

train,val=train_test_split(df_train,test_size=0.2,stratify=df_train.target,random_state=123)
vocab_size=len(vocabulary2)+1
max_len=40

word_index=get_word_index(vocabulary2)
### Chuẩn bị dữ liệu đã được xử lý
train_text=train['question_text_cleaned_2']
val_text=val['question_text_cleaned_2']
test_text=df_test['question_text_cleaned_2']

### mã hóa câu trong tập train sang dạng onehot cho dễ xử lý
encodes=fit_one_hot(word_index,train_text)
train_padded=pad_sequences(encodes,maxlen=max_len,padding="post")

### mã hóa câu trong tập validation sang dạng onehot cho dễ xử lý
encodes_=fit_one_hot(word_index,val_text)
val_padded=pad_sequences(encodes_,maxlen=max_len,padding="post")

### mã hóa câu trong tập test sang dạng onehot cho dễ xử lý
encodes__=fit_one_hot(word_index,test_text)
test_padded=pad_sequences(encodes__,maxlen=max_len,padding="post")

count=0

embedding_mat=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    try:
        vec=model_embed[word]
        embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue

print("Number of Out of Vocabulary",count)


# **4. Chuẩn bị Model**

**Mô hình LSTM**
![lstm](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

> LSTM cũng có kiến trúc dạng chuỗi như vậy, nhưng các mô-đun trong nó có cấu trúc khác với mạng RNN chuẩn. Thay vì chỉ có một tầng mạng nơ-ron, chúng có tới 4 tầng tương tác với nhau một cách rất đặc biệt

Sử dụng embedding layer, mục đích là để embedding sang một không gian mới có chiều nhỏ hơn, giảm chiều dữ liệu<br>
Bidirectional(LSTM) để xây model LSTM<br>
LSTM cũng là mạng CNN nên cần qua 2 lớp là Convo1D và Pool1D (convolution và pooling)

In [ ]:
def get_model_origin(matrix):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size,300,weights=[matrix],input_length=max_len,trainable=False)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Conv1D(64,3,activation="relu")(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    return model

opt=Adam(learning_rate=0.001)
BATCH_SIZE = 1024
bin_loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0, name='binary_crossentropy')

### Xác định điểm callback để giảm learning rate, và restore lại trọng số tốt nhất kề trước 
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=3,mode="min",restore_best_weights=True)

### Giảm learning rate khi model không được cải thiên (càng học càng ngu)
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.2, patience=2, verbose=1, mode="auto")

my_callbacks=[early_stopping,reduce_lr]

strategy = None

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Use TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('Use GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('Use CPU')

# **5. Huấn luyện và Dự đoán**

### **a. Thử nghiệm model 1**
Compile model với 30 epoch cho máu, batch_size để ở mức cao để tận dụng sức mạnh tính toán của TPU và sử dụng với tập validation để validate dữ liệu, truyền vào tham số callback để mô hình càng học càng khôn chứ không được ngu đi<br>
Một phần do em có cậu bạn cũng train với bọn Keras nhưng khi f1 score đạt được tốt rồi thì nó bắt đầu triệu chứng ngu đi khi mà loss tằng (overfitting)

In ra f1 score, ```f1_score``` tốt nhất ở đây là ```0.673``` tức ```67.3%``` ứng với hàm Preprocess thứ 2 tức là không đụng chạm nhiều đến câu hỏi<br>
Tương ứng là f1_score = ```0.633``` tức ```63.3%``` đối với hàm Preprocess đầu tiên khi loại bỏ ```stop_words```, ```lemma_words```, ...
Nhận xét:
- Với cách tiếp cận dựa trên pretrained thì việc tiền xử lý dữ liệu quá nhiều thực sự có ảnh hưởng lớn tới hiệu năng của model khi mà nó làm mất đi phần nhiều ý nghĩa các các từ có trong câu hỏi
- Có thể do cách cài đặt khác nhau nhưng về cơ bản với cách tiếp cận dựa vào pretrained thì ta không nên xử lý nhiễu của dữ liệu nhiều, có chăng là loại bỏ đi các digits, các công thức toán và các đường dẫn liên kết 

Có thể thấy được rằng mô hình hoạt động khá hiệu quả với tập **```train```** khi mà ```loss``` giảm đều và ```accuracy``` tăng đều qua các epoch<br>
Tuy nhiên thì với tập **```test```** lại hoạt động không được hiệu quả khi ```loss``` và ```accuracy``` có hình dạng trồi sụt nhìn như sóng biển. Và tệ hơn là với tập **```test```** thì ```loss``` về các epoch cuối lại tăng khá cao.<br>
Vậy thì có thể kết luận là do **model** đã xây dựng **hoạt động chưa hiệu quả**, cần phải xem xét lại<br>

In [ ]:
# THỬ NGHIỆM MODEL 1

with strategy.scope():
    google_model = get_model_origin(embedding_mat)
    google_model.compile(loss=bin_loss, optimizer=opt, metrics=['accuracy'])
history=google_model.fit(train_padded, train.target, batch_size=BATCH_SIZE, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

# summarize history for loss
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for accuracy
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

google_y_pre=google_model.predict(val_padded, verbose=1)
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(google_y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(google_y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(google_y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

del model_embed, history, best_score, best_thresh, google_model, google_y_pre
gc.collect()

In [ ]:
# THỬ NGHIỆM MODEL 2

def get_model(matrix):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, 300, weights=[matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.3)(x)
    x1 = Bidirectional(LSTM(256, return_sequences=True))(x)
    x2 = Bidirectional(GRU(128, return_sequences=True))(x1)
    max_pool1 = GlobalMaxPooling1D()(x1)
    max_pool2 = GlobalMaxPooling1D()(x2)
    conc = Concatenate()([max_pool1, max_pool2])
    predictions = Dense(1, activation='sigmoid')(conc)
    model = Model(inputs=inp, outputs=predictions)
    adam = optimizers.Adam(lr=0.001)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

strategy = None

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Use TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('Use GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('Use CPU')
        
with strategy.scope():
    google_model = get_model(embedding_mat)
    google_model.compile(loss=bin_loss, optimizer=Adam(lr=0.001), metrics=['accuracy'])
history=google_model.fit(train_padded, train.target, batch_size=BATCH_SIZE, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

# summarize history for loss
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for accuracy
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

google_y_pre=google_model.predict(val_padded, verbose=1)
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(google_y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(google_y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(google_y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

threshold=best_thresh
google_y_test_pre=google_model.predict(test_padded, batch_size=BATCH_SIZE, verbose=1)
google_y_test_pre=(google_y_test_pre>thresh).astype(int)

del embedding_mat, history, best_score, best_thresh
gc.collect()

# **6. So sánh, mở rộng**
Do sử dụng 1 pretrained là **Google News** nên kết quả thu được ở mức là **```0.68x``` (x<=3)** f1_score<br>
&nbsp;<br>
Bởi vì 3 thằng còn lại (không phải **Google News**) đều không ở dạng binary và không phải ở dạng chuẩn word_vector nên không đọc được bằng KeyedVectors, em sẽ định nghĩa một hàm để đọc các file embeddings còn lại <br>
Tuy nhiên do tự định nghĩa nên thời gian load dữ liệu vào biến khá là lâu

In [ ]:
def load_embed(file):
    def get_coefs(word,*arr):
        return word, np.asarray(arr, dtype='float32')

    if file == '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))

    return embeddings_index

!tree -h ./

glove_path = './glove.840B.300d/glove.840B.300d.txt'
paragram_path = './paragram_300_sl999/paragram_300_sl999.txt'
wiki_path = './wiki-news-300d-1M/wiki-news-300d-1M.vec'

glove_embed = load_embed(glove_path)
print("\033[1mGlove Coverage: \033[0m]")
oov_glove = check_voc(vocabulary2, glove_embed)

In [ ]:
# GLOVE

strategy = None

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Use TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('Use GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('Use CPU')
        
count=0
glove_embedding_mat=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    try:
        vec=glove_embed[word]
        glove_embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue

print("Number of Out of Vocabulary",count)

with strategy.scope():
    glove_model = get_model(glove_embedding_mat)
    glove_model.compile(loss=bin_loss, optimizer=Adam(lr=0.001), metrics=['accuracy'])
glove_history=glove_model.fit(train_padded, train.target, batch_size=BATCH_SIZE, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

# summarize history for loss
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 1)
plt.plot(glove_history.history['loss'])
plt.plot(glove_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for accuracy
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 2)
plt.plot(glove_history.history['accuracy'])
plt.plot(glove_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

glove_y_pre=glove_model.predict(val_padded, verbose=1)
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(glove_y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(glove_y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(glove_y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

threshold=best_thresh
glove_y_test_pre=glove_model.predict(test_padded, batch_size=BATCH_SIZE, verbose=1)
glove_y_test_pre=(glove_y_test_pre>thresh).astype(int)

del glove_embed, glove_embedding_mat, glove_history, best_score, best_thresh
gc.collect()

In [ ]:
# PARAGRAM
strategy = None

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Use TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('Use GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('Use CPU')
        

paragram_embed = load_embed(paragram_path)
print("\033[1mParagram Coverage: \033[0m]")
oov_paragram = check_voc(vocabulary2, paragram_embed)

count=0
para_embedding_mat=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    try:
        vec=paragram_embed[word.lower()]
        para_embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue

print("Number of Out of Vocabulary",count)

with strategy.scope():
    para_model = get_model(para_embedding_mat)
    para_model.compile(loss=bin_loss, optimizer=Adam(lr=0.001), metrics=['accuracy'])
para_history=para_model.fit(train_padded, train.target, batch_size=BATCH_SIZE, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

# summarize history for loss
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 1)
plt.plot(para_history.history['loss'])
plt.plot(para_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for accuracy
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 2)
plt.plot(para_history.history['accuracy'])
plt.plot(para_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

para_y_pre=para_model.predict(val_padded, verbose=1)
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(para_y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(para_y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(para_y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

threshold=best_thresh
para_y_test_pre=para_model.predict(test_padded, batch_size=BATCH_SIZE, verbose=1)
para_y_test_pre=(para_y_test_pre>thresh).astype(int)

del paragram_embed, para_embedding_mat, para_history, best_score, best_thresh
gc.collect()

In [ ]:
# WIKI 
strategy = None

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Use TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('Use GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('Use CPU')
        

wiki_embed = load_embed(wiki_path)
print("\033[1mWiki Coverage: \033[0m]")
oov_wiki = check_voc(vocabulary2, wiki_embed)

count=0
wiki_embedding_mat=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    try:
        vec=wiki_embed[word]
        wiki_embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue

print("Number of Out of Vocabulary",count)

with strategy.scope():
    wiki_model = get_model(wiki_embedding_mat)
    wiki_model.compile(loss=bin_loss, optimizer=Adam(lr=0.001), metrics=['accuracy'])
wiki_history=wiki_model.fit(train_padded, train.target, batch_size=BATCH_SIZE, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

# summarize history for loss
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 1)
plt.plot(wiki_history.history['loss'])
plt.plot(wiki_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for accuracy
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 2)
plt.plot(wiki_history.history['accuracy'])
plt.plot(wiki_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

wiki_y_pre=wiki_model.predict(val_padded, verbose=1)
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(wiki_y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(wiki_y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(wiki_y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

threshold=best_thresh
wiki_y_test_pre=wiki_model.predict(test_padded, batch_size=BATCH_SIZE, verbose=1)
wiki_y_test_pre=(wiki_y_test_pre>thresh).astype(int)

del wiki_embed, wiki_embedding_mat, wiki_history, best_score, best_thresh
gc.collect()

y_pre=0.25*(google_y_pre + glove_y_pre + para_y_pre + wiki_y_pre)
# y_pre=0.20*google_y_pre + 0.35*glove_y_pre + 0.15*para_y_pre + 0.30*wiki_y_pre
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

# y_test_pre = 0.25 * (google_y_test_pre + glove_y_test_pre + para_y_test_pre + wiki_y_test_pre)
y_test_pre = 0.2*google_y_test_pre + 0.3*glove_y_test_pre + 0.2*para_y_test_pre + 0.3*wiki_y_test_pre
y_test_pre=(y_test_pre>thresh).astype(int)
### Tạo File submission
submit=pd.DataFrame()
submit["qid"]=df_test.qid
submit["prediction"]=y_test_pre
submit.to_csv("submission.csv",index=False)
print("ok")